# **TRABALHO FINAL**

**REALIZE ANÁLISES EXPLORATÓRIAS E RESPONDA AS SEGUINTES PERGUNTAS:**

1. Qual(s) a(s) companhia(s) que mais registram atrasos? Analise qualitativamente e quantitativamente.
2. A rota ou aeronave podem influenciar nos atrasos?
3. Existe algum padrão ou tendência nos atrasos? Se sim, o que pode ser feito para reduzi-los?

Fonte de dados: https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv

In [ ]:
# Importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
# Lendo o dataframe
df = pd.read_csv('data/dados_voo.csv', index_col=0)
df.head()

Explicação de cada coluna:
- **year, month, day:** Date of departure broken in to year, month and day
- **dep_time, arr_time:** Departure and arrival times, format HMM or HHMM
- **dep_delay, arr_delay:** Departure and arrival delays, in minutes. Negative times represent early departures/arrivals.
- **hour, minute:** Time of departure broken in to hour and minutes
- **carrier:** Two letter carrier abbreviation
- **tailnum:** Plane tail number
- **flight:** Flight number
- **origin, dest:** Origin and destination.
- **air_time:** Amount of time spent in the air
- **distance:** Distance flown
- **data:** Date of departure in format yyyy-mm-dd

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

# 1. Qual(s) a(s) companhia(s) que mais registram atrasos? Analise qualitativamente e quantitativamente.

In [ ]:
# O atraso na saída e na chegada estão correlacionados? (dep_delay e arr_delay)
corr = df.loc[df['dep_delay'].notnull(), ['dep_delay', 'arr_delay']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlação dep_delay x arr_delay')
plt.show()

# Estão correlacionados!

In [ ]:
# Filtra APENAS quem saiu atrasado (dep_delay > 0)
voos_atrasados = df[df['dep_delay'] > 0]

# Desses atrasados, quantos chegaram no horário ou antes (arr_delay <= 0)?
recuperados = voos_atrasados[voos_atrasados['arr_delay'] <= 0].shape[0]

# Calcula a porcentagem real de recuperação
taxa_recuperacao = recuperados / voos_atrasados.shape[0]

print(f"Taxa de recuperação: {taxa_recuperacao:.2%}")

Podemos ver pela correção forte e pela taxa de recuperação de 27% que existe "gordura". As companhias aéreas mentem (no bom sentido). Por exemplo, elas dizem que o voo dura 2h, mas dura 1h45. Esse tempo extra permite que atrasos pequenos na saída (ex: 10 min) sejam absorvidos no ar. É por isso que podemos ver voos que saem atrasados e chegam adiantados. Porém, a correlação alta (0.91) prova que, quando o atraso na saída é grande, a “gordura” não aguenta e o atraso chega no destino.

In [ ]:
# Soma dos atrasos em minutos (da partida do vôo) por companhia, em termos absolutos
carrier_delays_absolute = df.groupby('carrier')['dep_delay'].sum().reset_index()
carrier_delays_absolute.sort_values('dep_delay', ascending=False)

In [ ]:
# Média de atrasos em minutos por companhia (partida do vôo), em termos relativos
num_delayed = (
    df[(df['dep_delay'].notnull()) & (df['dep_delay'] > 0)]
        .groupby('carrier')['dep_delay']
        .count()
)

num_flights = (
    df[df['dep_delay'].notnull()]
        .groupby('carrier')['dep_delay']
        .count()
)

carrier_delays_relative = (num_delayed / num_flights).reset_index()
carrier_delays_relative.sort_values('dep_delay', ascending=False, inplace=True)
carrier_delays_relative

In [ ]:
# Plotando os atrasos de vôo (valores absolutos)
plt.figure(figsize=(10, 4.5))
plt.title('Atrasos de vôo por companhia (valor absoluto)')
plt.xlabel('Companhia')
plt.ylabel('Soma de atrasos (min)')

ax = sns.barplot(
    data=carrier_delays_absolute,
    x='carrier',
    y='dep_delay',
    order=carrier_delays_absolute.sort_values('dep_delay', ascending=False)['carrier']
)
ax.set_yticklabels([f'{y:,.2f}' for y in ax.get_yticks()])

plt.tight_layout()
plt.show()

In [ ]:
# Plotando a proporção relativa de atrasos de vôo
plt.figure(figsize=(10, 4.5))
plt.title('Proporção de atrasos de vôo por companhia')
plt.xlabel('Companhia')
plt.ylabel('Porcentagem de atraso (de 0 a 1)')

ax = sns.barplot(
    data=carrier_delays_relative,
    x='carrier',
    y='dep_delay',
    order=carrier_delays_relative.sort_values('dep_delay', ascending=False)['carrier']
)
ax.set_yticklabels([f'{y:,.2f}' for y in ax.get_yticks()])

plt.tight_layout()
plt.show()

## Resposta

Em termos absolutos (maior minutagem total de atrasos, sem considerar a proporção em relação ao total de voos), as companhias com maiores atrasos são EV (1.024.829 minutos), B6 (705.417 minutos), UA (701.898 minutos), DL (442.482 minutos) e 9E (291.296 minutos).

Já em termos relativos (porcentagem de vôos com atrasos), as companhias com maiores atrasos são WN (54% de vôos com atraso), FL (51%), F9 (50%), UA (47%) E EV (45%)

# 2. A rota ou aeronave podem influenciar nos atrasos?

In [ ]:
# Quantidade de vôos mínimos para análise (tanto pras rotas quanto pros aviões)
min_flights = 100

## Rotas podem influenciar nos atrasos de vôo?

In [ ]:
# Criando a coluna de rota para auxiliar
df['route'] = df['origin'] + ' -> ' + df['dest']

In [ ]:
# Analisando a contagem, média e mediana das rotas com as piores médias (maior média de atrasos). Obs.: apenas rotas que tiveram mais de 100 vôos.
route_stats = df.groupby('route')['dep_delay'].agg(['count', 'mean', 'median'])

In [ ]:
route_stats[route_stats['count'] > min_flights].sort_values('mean', ascending=False)

In [ ]:
# Analisando média e mediana global das rotas
df['dep_delay'].agg(['mean', 'median', 'count'])

Nota: a média da pior rota é 41.8 (quase 4x a média global de 12.6). Como a mediana dela é baixa (7.5), isso significa que essa rota tem outliers muito grandes. Ela funciona bem na maioria das vezes, mas quando atrasa, atrasa muito.

In [ ]:
# Vendo os vôos com as piores medianas
route_stats[route_stats['count'] > min_flights].sort_values('median', ascending=False)

In [ ]:
# Top 50 rotas com maior frequência
tamanho_amostra = 50
top_routes = df['route'].value_counts().head(tamanho_amostra).index
df_top = df[df['route'].isin(top_routes)].copy().reset_index()

# Vendo o boxplot com outliers
plt.figure(figsize=(16, 6))
sns.boxplot(data=df_top, x='route', y='dep_delay')
plt.title(f'Atrasos de Chegada por Rota (Top {tamanho_amostra} mais frequentes)')
plt.xlabel('Rota')
plt.ylabel('Atraso de Chegada (minutos)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Plotando sem outliers pra visualizar melhor a localização das medianas
plt.figure(figsize=(16, 6))
sns.boxplot(
    data=df_top, 
    x='route', 
    y='dep_delay',
    showfliers=False,
)

plt.xticks(rotation=90)
# plt.ylim(-20, 40)
plt.tight_layout()
plt.show()

In [ ]:
# Analisando a varição (std) das medianas e outros valores
routes = df.groupby('route')['dep_delay'].agg(['median', 'count'])
routes_top = routes[routes['count'] > min_flights]
routes_top['median'].describe()

Analisando a mediana, vemos que a rota muda muito pouco a vida do passageiro. O desvio padrão das medianas é baixo (2.2) e a pior rota de todas tem mediana de 10 minutos. Ou seja, numa “viagem típica”, a diferença entre escolher a melhor ou a pior rota é de apenas uns 12 minutos (de -2 para 10). Isso é pouco relevante.

### Resposta sobre a influência da rota nos atrasos de vôo:
Sim, a rota influencia, mas atua principalmente em casos específicos, e não no atraso habitual.

Analisando rotas com volume relevante (>100 voos):


- Em situações típicas (mediana), a influência no atraso é baixa. O desvio padrão das medianas das rotas é de apenas 2.2 minutos, indicando que, em condições normais, a grande maioria das rotas opera com pontualidade similar (próxima a -2 minutos). Mesmo a rota com a pior mediana apresenta um atraso típico de apenas 10 minutos.

    
- Em situações atípicas (média), a influência é alta. As piores rotas apresentam médias de atraso até 3x superiores à média global (41.8 min vs 12.6 min), mesmo mantendo medianas baixas (7.5 min).


Ou seja, a rota não altera significativamente a probabilidade de um pequeno atraso corriqueiro, mas certas rotas específicas aumentam drasticamente a volatilidade e o risco de atrasos severos. 

## Aviões podem influenciar nos atrasos de vôo?

Ao comparar a média geral dos atrasos e comparar com as médias mais altas de atraso entre os aviões com maior frequência de vôo, podemos pensar que há aviões que de fato influenciam no tempo do atraso, pois eles se distanciam muito da média global.

In [ ]:
# Analisando a contagem, média e mediana dos aviões com as piores médias (maior média de atrasos). Obs.: apenas aviões que tiveram mais de 100 vôos.
aviao_stats = df.groupby('tailnum')['dep_delay'].agg(['count', 'mean', 'median'])
aviao_stats[aviao_stats['count'] > min_flights].sort_values('median', ascending=False).head()

In [ ]:
# Atraso médio geral
df['dep_delay'].agg(['mean', 'median'])

Porém, a média é sensível a outliers. Ao plotar o boxplot das variáveis mais frequentes, percebemos que as ‘caixas’ (onde estão 50% dos voos, o comportamento padrão) estão alinhadas. Os pontos fora da curva (outliers) é que puxavam a média para cima. Veja abaixo:

In [ ]:
# Top 70 aeronaves por frequência
tamanho_amostra = 70
top_tailnums = df['tailnum'].value_counts().head(tamanho_amostra).index
df_top = df[df['tailnum'].isin(top_tailnums)].copy().reset_index()

# Boxplot com seaborn (mais limpo)
plt.figure(figsize=(16, 6))
sns.boxplot(data=df_top, x='tailnum', y='dep_delay')
plt.title(f'Atrasos de Chegada por Aeronave (Top {tamanho_amostra} mais frequentes)')
plt.xlabel('Número da Aeronave')
plt.ylabel('Atraso de Chegada (minutos)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Plotando sem os outliers para visualizar melhor onde está localizada as medianas
plt.figure(figsize=(16, 6))
sns.boxplot(
    data=df_top, 
    x='tailnum', 
    y='dep_delay',
    showfliers=False,               
    showcaps=False,                 
    whiskerprops={'visible': False} 
)

plt.xticks(rotation=90)
plt.ylim(-20, 40)
plt.tight_layout()
plt.show()

Como queremos entender o comportamento típico da aeronave e não puni-la por eventos isolados (como uma tempestade que atrasou um único voo em 5 horas), adotamos a mediana como régua.

In [ ]:
# Analisando a varição (std) das medianas e outros valores
airplane = df.groupby('tailnum')['dep_delay'].agg(['median', 'count'])
airplane_top = airplane[airplane['count'] > min_flights]
airplane_top['median'].describe()

### Resposta sobre a influência do aviões nos atrasos de vôo:
Concluímos que a aeronave específica não exerce influência significativa nos atrasos. A variação de comportamento entre um avião e outro é desprezível (menos de 2 minutos de variação típica). Os grandes atrasos observados na média são eventos pontuais, não estruturais da aeronave.

# 3. Existe algum padrão ou tendência nos atrasos? Se sim, o que pode ser feito para reduzi-los?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))

# Média de atraso por hora do dia
hora_stats = df.groupby('hour')['dep_delay'].mean()
hora_stats.plot(ax=ax[0], marker='o', linestyle='-')
ax[0].set_title('Tendência de atraso por hora do dia')
ax[0].set_ylabel('Média de atraso (min)')
ax[0].grid(True, alpha=0.3)

# Média de atraso por mês
mes_stats = df.groupby('month')['dep_delay'].mean()
mes_stats.plot(ax=ax[1], marker='o', color='orange')
ax[1].set_title('Sazonalidade dos atrasos (por mês)')
ax[1].set_xticks(range(1, 13)) # Garante que mostre meses 1 a 12
ax[1].grid(True, alpha=0.3)
ax[1].set_ylim(0)

plt.show()

### Resposta

Sim, existem padrões temporais claros e previsíveis.

Analisando os dados agregados, podemos identificar duas tendências principais que regem os atrasos:
1. Acumulação diária (Efeito Cascata): Os dados mostram uma progressão linear de atrasos ao longo do dia. A operação inicia-se pontual (até adiantada) às 5h da manhã, mas a média de atraso sobe consistentemente a cada hora, atingindo seus picos à noite. Isso indica que o sistema não tem “folga” suficiente para absorver pequenos atrasos, que se acumulam e propagam para os voos seguintes.<br><br>**Possível solução**: aumentar o tempo de solo (turnaround time) programado entre voos, especialmente no período da tarde, criando “janelas de recuperação” para zerar o efeito cascata antes que ele se torne crítico à noite.<br><br>

2. Sazonalidade anual: Observa-se uma elevação significativa nos atrasos durante os meses de verão (Junho/Julho) e Dezembro. Esses picos coincidem com alta temporada e períodos de clima adverso.<br><br>**Possível solução**: Ajuste preventivo da malha aérea nesses meses específicos, reduzindo a quantidade de voos para diminuir o congestionamento do espaço aéreo e alocando aeronaves de reserva para cobrir imprevistos operacionais.